In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from arsenal import colors
from bench.spider.interface import SpiderInterface

import nest_asyncio
nest_asyncio.apply()

In [ ]:
spider = SpiderInterface()

In [ ]:
for x in spider.dev_data[:2]:
    print()
    print(x.text)
    print(x.gold_sql)
    print(x.db_name)

In [ ]:
x = spider.dev_data[0]
x

In [ ]:
x.evaluate(x.gold_sql)

In [ ]:
x = spider.dev_data[1]
x

In [ ]:
x.evaluate('SELECT count(*) FROM singer')

In [ ]:
x.run_query('SELECT count(*) FROM singer')

In [ ]:
from pathlib import Path

import bench.spider.evaluation as E
from bench.spider.evaluation import (
    #build_foreign_key_map_from_json,
    build_valid_col_units,
    rebuild_sql_val,
    rebuild_sql_col,
    eval_exec_match,
)

def evaluate(self, gold: str, pred: str, db_name: str):
    """Returns: bool, Optional[str]

    On success (i.e., predicted execution result is the same as gold), returns `(True, None)`
    On failure, returns `(False, reason)` where reason is one of the two cases:
    * `invalid` if `pred` sql is not a well-formed sql statement that can be parsed by sqlite
    * `mismatch` if `pred` is a well-formed sql but the execution result is different from that of the `gold`.
    """
    db = self.db_path / db_name / (db_name + '.sqlite')
    schema = E.Schema(E.get_schema(db))
    g_sql = E.get_sql(schema, gold)

    try:
        p_sql = E.get_sql(schema, pred)
    except Exception as e:
        # sql is ill-formed (can't be parsed by sqlite engine)
        print(colors.red % e.__class__.__name__, e, )

        import traceback
        traceback.print_exc()
        
        return False, 'invalid'

    kmap = self.kmaps[db_name]
    g_valid_col_units = build_valid_col_units(g_sql['from']['table_units'], schema)
    g_sql = rebuild_sql_val(g_sql)
    g_sql = rebuild_sql_col(g_valid_col_units, g_sql, kmap)
    p_valid_col_units = build_valid_col_units(p_sql['from']['table_units'], schema)
    p_sql = rebuild_sql_val(p_sql)
    p_sql = rebuild_sql_col(p_valid_col_units, p_sql, kmap)

    exec_match = eval_exec_match(db, pred, gold, p_sql, g_sql)
    reason = None if exec_match else 'mismatch'

    return exec_match, reason

#spider.evaluator = evaluate

In [ ]:
#x.interface.evaluator = evaluate

In [ ]:
x.interface.evaluator.evaluate = evaluate

In [ ]:
junk_sql = 'select count(*) from singer'

In [ ]:
x.interface.evaluator.evaluate(x.interface.evaluator, gold = x.gold_sql, pred = junk_sql, db_name = x.db_name)

In [ ]:
#x.db_schema.columns

In [ ]:
grammar_text = open('/home/timv/projects/genparse/benchmark/grammars/sql_case_insensitive.lark').read()

In [ ]:
from genparse.util import InferenceSetupVLLM, InferenceSetup

In [ ]:
#infer = InferenceSetup('codellama', grammar_text, proposal_name='character', guide_opts={'ignore': '\s*'})
infer = InferenceSetupVLLM('codellama', grammar_text, proposal_name='character', guide_opts={'ignore': r'\s*'}, batch_size=50)

In [ ]:
infer.sampler.llm._model.batch_size = 80

In [ ]:
p = infer(x.describe(), n_particles=50, method='smc-standard', max_tokens=40)

In [ ]:
p[0]

In [ ]:
x = spider.dev_data[0]

In [ ]:
x.text

In [ ]:
for y, py in sorted(p[0].posterior.items(), key=lambda ab: -ab[1]):
    y = y[:-1]  # remove EOS
    print(f'{colors.mark(x.evaluate(y))} {py:-.6f} {y}')
    try:
        display(x.run_query(y))
    except Exception as e:
        print()
        print(colors.dark.red % '💀 ERROR', e)
        print()
